## Step 1 Import

In [4]:
import torch 
import json
from datasets import *
import numpy as np
from datasets import load_dataset
from transformers import Trainer, TrainingArguments,  DataCollatorForLanguageModeling, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM, AdamW, get_scheduler
import pandas as pd
from rouge import Rouge

/home/jie/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2 Datasets

In [38]:
class MedicalDialogueDataset(Dataset):
    def __init__(self, split, percent=100, seed=42):  # 添加seed参数，默认值为42
        # Load data set
        ds = load_dataset("omi-health/medical-dialogue-to-soap-summary", split=split)
        
        # Remove unwanted columns
        columns_to_remove = ['messages', 'prompt']
        ds = ds.remove_columns(columns_to_remove)
        
        # Replace line breaks and rename columns
        ds = ds.rename_column('soap', 'summary')
        
        # Add the ID and format the digest
        ds = ds.map(self.add_id, with_indices=True)
        ds = ds.map(self.format_summary)
        
        # Select the corresponding percentage
        if percent < 100:
            ds = ds.shuffle(seed=seed).select(range(int(percent / 100.0 * len(ds))))
        
        self.data = ds
    
    def add_id(self, example, idx):
        example['id'] = str(idx)
        return example
    
    def format_summary(self, example):
        example['summary'] = example['summary'].replace('S: ', 'Subjective: ')
        example['summary'] = example['summary'].replace('O: ', 'Objective: ')
        example['summary'] = example['summary'].replace('A: ', 'Assessment: ')
        example['summary'] = example['summary'].replace('P: ', 'Plan: ')
        return example

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]  # 获取索引对应的数据项
        ordered_item = {'id': item['id']}  # 创建一个新字典，并首先加入'id'
        ordered_item.update({k: item[k] for k in item if k != 'id'})  # 添加其他字段，排除'id'
        # return item['dialogue'], item['summary'] 
        return ordered_item



In [6]:

# Create training dataset instances of different percentages
train_data = MedicalDialogueDataset('train', percent=20, seed=42)
# train_data = MedicalDialogueDataset('train', percent=50, seed=42)
# train_data = MedicalDialogueDataset('train', percent=100, seed=42)  

valid_data = MedicalDialogueDataset('validation')
test_data = MedicalDialogueDataset('test')

Map: 100%|██████████| 250/250 [00:00<00:00, 19958.05 examples/s]


In [8]:
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))

train set size: 1850
valid set size: 500
test set size: 250
{'id': '8647', 'dialogue': "Doctor: Good morning, how can I help you today?\nPatient: Hi doctor, I recently underwent an abdominal ultrasonography (USG) for my bilateral renal nephrolithiasis.\nDoctor: I see. Tell me about your general health. How is your blood biochemistry, and do you have any cardiovascular or hormonal disorders?\nPatient: My blood biochemistry is normal, and I don't have any cardiovascular or hormonal disorders. I had an operation 17 years ago to repair my extrophic bladder, and they created an Indiana pouch for me.\nDoctor: Alright. Can you tell me about your weight and body mass index (BMI)?\nPatient: My weight is 85 kg, and my BMI is 28.7 kg/m2.\nDoctor: Thank you for the information. Now, let's talk about your USG results. It showed a hyperechogenic lesion at the fat intensity, filling out your right renal sinus completely. A computerized tomography (CT) scan confirmed the presence of a fatty mass that 

## Step 3 Data Preprocessing

Choose one of the following models to run

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


### bart-large-text-summarizer

In [13]:
max_input_length = 512
max_target_length = 256

tokenizer = AutoTokenizer.from_pretrained("Azma-AI/bart-large-text-summarizer")
model = AutoModelForSeq2SeqLM.from_pretrained("Azma-AI/bart-large-text-summarizer")
# Transfer the model to the GPU and convert to semi-precision
model = model.to(device).half()
optimizer = AdamW(model.parameters(), lr=5e-5)


/home/jie/.local/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### bart-large-xsum-samsum

In [ ]:
max_input_length = 512
max_target_length = 256
tokenizer = AutoTokenizer.from_pretrained("lidiya/bart-large-xsum-samsum")
model = AutoModelForSeq2SeqLM.from_pretrained("lidiya/bart-large-xsum-samsum").to(device).half()
optimizer = AdamW(model.parameters(), lr=5e-5)

### ssr-base-finetuned-samsum-en

In [ ]:
max_input_length = 512
max_target_length = 256
tokenizer = AutoTokenizer.from_pretrained("santiviquez/ssr-base-finetuned-samsum-en")
model = AutoModelForSeq2SeqLM.from_pretrained("santiviquez/ssr-base-finetuned-samsum-en").to(device).half()
optimizer = AdamW(model.parameters(), lr=5e-5)

### T5-Finetuned-Summarization-DialogueDataset

In [ ]:
max_input_length = 512
max_target_length = 256
tokenizer = AutoTokenizer.from_pretrained("gauravkoradiya/T5-Finetuned-Summarization-DialogueDataset")
model = AutoModelForSeq2SeqLM.from_pretrained("gauravkoradiya/T5-Finetuned-Summarization-DialogueDataset").half()
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

In [15]:
def collote_fn(batch_samples):
    batch_inputs, batch_targets = [], []
    for sample in batch_samples:
        batch_inputs.append(sample['dialogue'])
        batch_targets.append(sample['summary'])
    batch_data = tokenizer(
        batch_inputs, 
        padding=True, 
        max_length=max_input_length,
        truncation=True, 
        return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch_targets, 
            padding=True, 
            max_length=max_target_length,
            truncation=True, 
            return_tensors="pt"
        )["input_ids"]
        batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(labels)
        end_token_index = torch.where(labels == tokenizer.eos_token_id)[1]
        for idx, end_idx in enumerate(end_token_index):
            labels[idx][end_idx+1:] = -100
        batch_data['labels'] = labels
    return batch_data

In [16]:
train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=4, shuffle=False, collate_fn=collote_fn)
test_dataloader = DataLoader(test_data, batch_size=4,shuffle=False)

Evaluate the data set before fine-tuning

In [17]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def predict_summary(dialogue, model, tokenizer):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(dialogue, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
        outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=512)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

results = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for batch in tqdm(test_dataloader, desc="Generating Summaries"):
    dialogues = batch['dialogue']
    reference_summaries = batch['summary']
    
    for dialogue, reference_summary in zip(dialogues, reference_summaries):
        predicted_summary = predict_summary(dialogue, model, tokenizer)
        scores = scorer.score(reference_summary, predicted_summary)
        results.append({
            "Dialogue": dialogue,
            "Reference Summary": reference_summary,
            "Predicted Summary": predicted_summary,
            "ROUGE-1": scores['rouge1'].fmeasure,
            "ROUGE-2": scores['rouge2'].fmeasure,
            "ROUGE-L": scores['rougeL'].fmeasure
        })

        # Accumulate scores
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Calculate the average ROUGE score
average_scores = {key: sum(values) / len(values) for key, values in rouge_scores.items()}
print("Average ROUGE Scores:", average_scores)

# Save to JSON file
with open('pre_bart_large_text_summarizer.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("Pre-training predictions and ROUGE scores saved successfully.")

Generating Summaries: 100%|██████████| 63/63 [02:16<00:00,  2.17s/it]

Average ROUGE Scores: {'rouge1': 0.25684714494565014, 'rouge2': 0.12714116561972183, 'rougeL': 0.18907099419037057}
Pre-training predictions and ROUGE scores saved successfully.


Convert the JSON file to a csv file

In [20]:
model_name = 'pre_bart_large_text_summarizer'
# model_name = 'pre_bart_large_xsum_samsum'
# model_name = 'pre_ssr_base_finetuned_samsum'
# model_name = 'pre_T5_finetuned_summarization_DialogueDataset'
results = []
for idx in range(len(test_data)):
    try:
        dialogue, reference_summary = test_data[idx]
        predicted_summary = predict_summary(dialogue, model, tokenizer)
        results.append({
            "Dialogue": dialogue,
            "Reference Summary": reference_summary,
            "Predicted Summary": predicted_summary
        })
    except ValueError as e:
        print(f"Error at index {idx}: {e}")


# 保存到CSV
df = pd.DataFrame(results)
df.to_csv(f"{model_name}.csv", index=False)

Since the AutoModelForSeq2SeqLM function that comes with the Transformers library is used to build the model directly here, I process the data in each batch into a format acceptable to the model: A dictionary containing the keys 'attention_mask', 'input_ids', 'labels' and 'decoder_input_ids'

In [21]:
batch = next(iter(train_dataloader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print(batch)

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'])
batch shape: {'input_ids': torch.Size([4, 512]), 'attention_mask': torch.Size([4, 512]), 'decoder_input_ids': torch.Size([4, 256]), 'labels': torch.Size([4, 256])}
{'input_ids': tensor([[    0, 41152,    35,  ...,    83,   314,     2],
        [    0, 41152,    35,  ...,     1,     1,     1],
        [    0, 41152,    35,  ...,  1848,   783,     2],
        [    0, 41152,    35,  ...,     8,    21,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'decoder_input_ids': tensor([[    2,     0, 47159,  ...,  5929, 44030,  4866],
        [    2,     0, 47159,  ...,  1233,  3855,    11],
        [    2,     0, 47159,  ...,   119,  3964, 44704],
        [    2,     0, 47159,  ..., 27225,  7168,   618]]), 'labels': tensor([[    0, 47159,  2088,  ..., 44030,  4866,     2],
        [    0, 47159,  2088,  ...,

/home/jie/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Step 4 Fine-Tuning

The model constructed by AutoModelForSeq2SeqLM has been packaged with the corresponding loss function, and the calculated loss will be directly included in the outputs of the model, which can be obtained directly through outputs.loss, so the training cycle is as follows:

In [22]:

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)
    
    model.train()
    for batch, batch_data in enumerate(dataloader, start=1):
        batch_data = batch_data.to(device)
        outputs = model(**batch_data)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
    return total_loss

In the validation/test loop, the prediction is first obtained through the model.generate() function, and then both the prediction and the correct label are processed into the text list format accepted by the rouge library (here I replaced the -100 in the tag sequence with the pad token ID for easy decoding by the word separator). Finally, it is sent to the rouge library to calculate each ROUGE value:

In [24]:

rouge = Rouge()

def test_loop(dataloader, model):

    preds, labels = [], []
    
    model.eval()
    for batch_data in tqdm(dataloader):
        # batch_data = batch_data.to(device)
        batch_data = {k: v.to(device) for k, v in batch_data.items()}
        with torch.no_grad():
            generated_tokens = model.generate(
                batch_data["input_ids"],
                attention_mask=batch_data["attention_mask"],
                max_length=max_target_length,
                num_beams=4,
                no_repeat_ngram_size=2,
            ).cpu().numpy()
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        preds += [' '.join(pred.strip()) for pred in decoded_preds]
        labels += [' '.join(label.strip()) for label in decoded_labels]
    scores = rouge.get_scores(hyps=preds, refs=labels, avg=True)
    result = {key: value['f'] * 100 for key, value in scores.items()}
    result['avg'] = np.mean(list(result.values()))
    print(f"Rouge1: {result['rouge-1']:>0.2f} Rouge2: {result['rouge-2']:>0.2f} RougeL: {result['rouge-l']:>0.2f}\n")
    return result

In [25]:

learning_rate = 1e-7
epoch_num = 10

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_avg_rouge = 0.
for t in range(epoch_num):
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, optimizer, lr_scheduler, t+1, total_loss)
    valid_rouge = test_loop(valid_dataloader, model)
    print(valid_rouge)
    rouge_avg = valid_rouge['avg']
    if rouge_avg > best_avg_rouge:
        best_avg_rouge = rouge_avg
        print('saving new weights...\n')
        torch.save(model.state_dict(), f'epoch_{t+1}_valid_rouge_{rouge_avg:0.4f}_model_weights.bin')
print("Done!")

Epoch 1/10
-------------------------------


100%|██████████| 125/125 [05:13<00:00,  2.51s/it]


Rouge1: 89.48 Rouge2: 73.70 RougeL: 86.16

{'rouge-1': 89.47554793687289, 'rouge-2': 73.69927137313994, 'rouge-l': 86.15641815026342, 'avg': 83.11041248675875}
saving new weights...

Epoch 2/10
-------------------------------


100%|██████████| 125/125 [05:29<00:00,  2.64s/it]


Rouge1: 89.62 Rouge2: 73.58 RougeL: 86.15

{'rouge-1': 89.62083377597432, 'rouge-2': 73.57828468062392, 'rouge-l': 86.15466260770117, 'avg': 83.11792702143313}
saving new weights...

Epoch 3/10
-------------------------------


...

Rouge1: 91.90 Rouge2: 77.26 RougeL: 88.80

{'rouge-1': 91.90016703860374, 'rouge-2': 77.26398998120823, 'rouge-l': 88.79812266692937, 'avg': 85.98742656224711}
Epoch 9/10
-------------------------------


100%|██████████| 125/125 [05:09<00:00,  2.48s/it]


Rouge1: 92.09 Rouge2: 77.26 RougeL: 88.74

{'rouge-1': 92.09392402074845, 'rouge-2': 77.26008367393649, 'rouge-l': 88.73735772338159, 'avg': 86.03045513935551}
Epoch 10/10
-------------------------------


100%|██████████| 125/125 [05:10<00:00,  2.49s/it]


Rouge1: 92.10 Rouge2: 77.44 RougeL: 88.71

{'rouge-1': 92.10007865979176, 'rouge-2': 77.43745142714089, 'rouge-l': 88.71038435666864, 'avg': 86.0826381478671}
Done!


## Step 5 Test model 

After training, we load the model weights that perform best on the validation set, report their performance on the test set, and save the model predictions to a file.

Since AutoModelForSeq2SeqLM encapsulates the entire decoding process, we just need to call generate() function to automatically find the best token ID sequence through beam search, Therefore, at last, you only need to convert the token ID sequence into text using the word divider to obtain the generated summary:

In [40]:
test_dataloader = DataLoader(test_data, batch_size = 4, shuffle = False, collate_fn = collote_fn)
model.load_state_dict(torch.load('epoch_7_valid_rouge_86.2480_model_weights.bin'))

<All keys matched successfully>

In [41]:
model.eval()

with torch.no_grad():
    print('evaluating on test set...')
    sources, preds, labels = [], [], []
    for batch_data in tqdm(test_dataloader):
        batch_data = batch_data.to(device)
        generated_tokens = model.generate(
            batch_data["input_ids"],
            attention_mask=batch_data["attention_mask"],
            max_length=max_target_length,
            num_beams=4,
            no_repeat_ngram_size=2,
        )

        generated_tokens = generated_tokens.cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_sources = tokenizer.batch_decode(
            batch_data["input_ids"].cpu().numpy(), 
            skip_special_tokens=True
        )
        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        sources += [source.strip() for source in decoded_sources]
        preds += [pred.strip() for pred in decoded_preds]
        labels += [label.strip() for label in decoded_labels]
    scores = rouge.get_scores(
        hyps=[' '.join(pred) for pred in preds], 
        refs=[' '.join(label) for label in labels], 
        avg=True
    )
    rouges = {key: value['f'] * 100 for key, value in scores.items()}
    rouges['avg'] = np.mean(list(rouges.values()))
    print(f"Test Rouge1: {rouges['rouge-1']:>0.2f} Rouge2: {rouges['rouge-2']:>0.2f} RougeL: {rouges['rouge-l']:>0.2f}\n")
    results = []
    print('saving predicted results...')
    for source, pred, label in zip(sources, preds, labels):
        results.append({
            "document": source, 
            "prediction": pred, 
            "summarization": label
        })
    with open('post_bart_large_text_summarizer.json', 'wt', encoding='utf-8') as f:
        for exapmle_result in results:
            f.write(json.dumps(exapmle_result, ensure_ascii=False) + '\n')



evaluating on test set...


100%|██████████| 63/63 [02:53<00:00,  2.75s/it]


Test Rouge1: 91.87 Rouge2: 77.62 RougeL: 88.52

saving predicted results...


The fine-tuned inference produced by the json file is saved as a csv file.

In [44]:
import pandas as pd
model_name = 'bart-large-text-summarizer'

results = []
for idx in range(len(test_data)):
    try:
        dialogue, reference_summary = test_data[idx]
        predicted_summary = predict_summary(dialogue, model, tokenizer)
        results.append({
            "Dialogue": dialogue,
            "Reference Summary": reference_summary,
            "Predicted Summary": predicted_summary
        })
    except ValueError as e:
        print(f"Error at index {idx}: {e}")


# 保存到CSV
df = pd.DataFrame(results)
df.to_csv(f"post_{model_name}.csv", index=False)

Remove a piece of data for viewing

In [42]:
def predict_summary(input_text, model, tokenizer, device='cuda'):
    model.to(device)
    model.eval()

    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    inputs = inputs.to(device)

    outputs = model.generate(
        inputs["input_ids"],
        max_length=512,
        num_beams=10,
        no_repeat_ngram_size=2,
        early_stopping=False
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)


    return summary

# 使用函数
input_text = "Doctor: Hello, I remember you had an emergency caesarean delivery at 39 weeks due to fetal distress. How have you been since then? Any postpartum complications? Patient: Hi, Doctor. I've been doing well since the delivery. No complications, thankfully. Doctor: That's good to hear. As part of our ongoing study on 'Vaginal delivery after caesarean section', you underwent a saline contrast sonohysterography 6 months after the caesarean section. The results showed a small indentation in your caesarean scar, and the remaining myometrium over the defect was 7.5 mm (Fig. ). Patient: Oh, I see. What does that mean for my current pregnancy? Doctor: At around 11 weeks, you had a dating scan with no remarks. Then, you came for a transvaginal ultrasound examination at around 13 weeks asc part of our study. The scan revealed a duplex pregnancy with one viable intrauterine fetus with normal anatomy and placenta located high on the anterior wall. A small gestational sac (8 mm) with a yolk sac without an embryo was located in the caesarean scar (Fig. ). There was no extensive vascularity surrounding the sac, and you were asymptomatic. Patient: Yes, that's right. I didn't feel any discomfort or symptoms. Doctor: We informed you that there wasn't enough evidence to advise a specific management for this condition. After discussion with you and your husband, expectant management was chosen with a new ultrasound examination scheduled after 5 weeks. Patient: Yes, we decided to wait and see how things would progress. Doctor: You came to our ultrasound department at 18 weeks, 22 weeks, and 30 weeks of gestation. Throughout this time, you remained asymptomatic. The ectopic gestational sac was not visualized with transvaginal or transabdominal scans at the 18 weeks examination (Fig. ). The niche in the scar and the thickness of the thinnest part of the remaining myometrium appeared unchanged at all visits. Patient: That's a relief. How's the intrauterine pregnancy developing? Doctor: The intrauterine pregnancy developed normally with no signs of abnormal placentation. At 30 weeks of gestation, the ultrasound appearance of the scar area did not indicate any contraindications for vaginal delivery. The thickness of the lower uterine segment (LUS) was 4.9 mm (Fig. ). Patient: So, I can have a vaginal delivery this time? Doctor: Yes, in agreement with you, we've planned for a vaginal delivery. The staff of the labor ward has been fully informed and prepared for your case. Patient: That's great news! Thank you, Doctor. Doctor: You're welcome. You'll be admitted to the labor ward when the time comes. Please continue to monitor your symptoms and reach out if you have any concerns. Good luck with the rest of your pregnancy. Patient: Thank you so much, Doctor. I appreciate your help and guidance throughout this process."
summary = predict_summary(input_text, model, tokenizer, device)
print(summary)

Subjective: The patient, who had an emergency caesarean delivery at 39 weeks due to fetal distress, reports no postpartum complications. She underwent saline contrast sonohysterography 6 months post-cavear section, revealing a small indentation in the scar and a remaining myometrium of 7.5 mm. At 11 weeks, she underwent a transvaginal ultrasound, which showed a duplex pregnancy with one viable intrauterine fetus and normal anatomy and placenta. A small gestational sac (8 mm) with a yolk sac without an embryo was noted, but there was no extensive vascularity surrounding the sac. During subsequent ultrasounds at 18, 22, and 30 weeks of gestation, the ectopic sac was not visualized. No contraindications for vaginal delivery were noted. The thickness of the lower uterine segment (LUS) was 4.9 mm (Fig.
